In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from fbprophet import Prophet
import math
import os
import glob
%matplotlib inline

package_dir = os.getcwd()

print(package_dir)


/home/sebastian/Programming/Bachelorthesis


In [83]:
df = pd.DataFrame()

print('Loading CSVs\n')
for file in glob.glob('res/*.csv'):
    # read single file, index on StationEoI and DatetimeEnd
    read = pd.read_csv(file,
                 encoding="utf-16", parse_dates=[13, 14],
                 infer_datetime_format=True,
                 index_col=[4, 14])
    # drop 'bulk' files because they have different averaging
    bulks = read.SamplingPoint.str.lower().str.contains('bulk')
    clean = read[~bulks].copy()
    
    # ignore unnecessary columns
    clean.drop(columns=['Countrycode', 'Namespace', 'AirQualityNetwork',
                 'AirQualityStation', 'SamplingPoint', 'Sample',
                 'SamplingProcess', 'AirPollutantCode',
                 'DatetimeBegin', 'Validity', 'Verification',
                 'AveragingTime'],
        inplace=True)
    
    # make pollutant a column for better memory usage
    clean = clean.pivot(columns='AirPollutant')
    df = pd.concat([df, clean], axis=1)

# make names shorter    
df.index.names = ['StationEoI', 'Timestamp']
df.columns.names = [None, 'Pollutant']

print('\nFinished Loading')
print('Sorting')
df = df.sort_index()
print('Finished sorting')

Loading CSVs



/usr/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



Finished Loading
Sorting


Finished sorting


In [50]:
df = pd.DataFrame()

for file in glob.glob('res/*2013_timeseries.csv'):
    # read single file, index on StationEoI and DatetimeEnd
    read = pd.read_csv(file,
                 encoding="utf-16", parse_dates=[13, 14],
                 infer_datetime_format=True)
    # drop 'bulk' files because they have different averaging
    bulks = read.SamplingPoint.str.lower().str.contains('bulk')
    clean = read[~bulks].copy()
    
    # ignore unnecessary columns
    clean.drop(columns=['Countrycode', 'Namespace', 'AirQualityNetwork',
                 'AirQualityStation', 'SamplingPoint', 'Sample',
                 'SamplingProcess', 'AirPollutantCode',
                 'DatetimeBegin', 'Validity', 'Verification',
                 'AveragingTime'],
        inplace=True)
    
    df = pd.concat([df, clean], ignore_index=True)

# make pollutant a column for better memory usage
#df = df.pivot(columns='AirPollutant')

# make names shorter    
#df.index.names = ['StationEoI', 'Timestamp']
#df.columns.names = [None, 'Pollutant']

In [ ]:
df[df.duplicated()]

0         False
1         False
2         False
3         False
4         False
5         False
6         False
7         False
8         False
9         False
10        False
11        False
12        False
13        False
14        False
15        False
16        False
17        False
18        False
19        False
20        False
21        False
22        False
23        False
24        False
25        False
26        False
27        False
28        False
29        False
          ...  
122670    False
122671    False
122672    False
122673    False
122674    False
122675    False
122676    False
122677    False
122678    False
122679    False
122680    False
122681    False
122682    False
122683    False
122684    False
122685    False
122686    False
122687    False
122688    False
122689    False
122690    False
122691    False
122692    False
122693    False
122694    False
122695    False
122696    False
122697    False
122698    False
122699    False
Length: 122700, dtype: b

In [56]:
print(df.head())

  AirQualityStationEoICode AirPollutant  Concentration UnitOfMeasurement  \
0                  DESN077   As in PM10          2.284             ng/m3   
1                  DESN077   As in PM10          1.489             ng/m3   
2                  DESN077   As in PM10          3.764             ng/m3   
3                  DESN077   As in PM10          0.253             ng/m3   
4                  DESN077   As in PM10          1.836             ng/m3   

          DatetimeEnd  
0 2013-02-18 23:00:00  
1 2013-03-26 23:00:00  
2 2013-05-07 23:00:00  
3 2013-11-03 23:00:00  
4 2013-12-03 23:00:00  


In [54]:
df.shape

(122700, 5)